# Очередь с приоритетом

Очередь с приритетом - структура данных, которая позволяет определить самый приоритетный элемент в множестве (по заданному критерию), а также добавлять и удалять оттуда элементы (в особенности - приоритетный элемент). 

# Наивный подход

Самым очевидным подходом к построению очередей с приоритетом является отсортированные последовательности. Действительно, элементы, отсортированные по критерию, будут представлять собой очередь с приоритетом - первым элементом будет являться самый приоритетный, а за ним все остальные по степени важности.

In [1]:
#include <iostream>
#include <vector>
#include <algorithm>
#include <set>
#include <functional>

In [2]:
std::vector<int> v1({4, 8, 1, 2, 8, 3});

(std::vector<int> &) { 4, 8, 1, 2, 8, 3 }


In [3]:
std::sort(v1.begin(), v1.end(), [](int i, int j){ return i > j; /*критерий*/}); // построение

(void) nullptr


In [4]:
std::cout << "Самый приоритетный: " << v1.front();

Самый приоритетный: 8

(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa6da4bd6e0


Однако такой подход не является эффектиный по производительности:

Стоимость построения очереди из набора данных составляет в лучшем случае $n \ln n$ (лучшее время для сортировки).

Стоимость добавления нового элемента будет составлять время на поиск места вставки - $n$ (линейный поиск), $\ln n$ (бинарный поиск) + время на вставку - $n$ = $\ln n + n$.

Стоимость нахождения самого приоритетного элемента будет состовлять всего 1 - нужно просто взять первый элемент.

# Бинарные деревья поиска

Более рациональным подходом будет являться использование деревьев поиска. 

Стоимость построения из набора данных все также будет равняться $n \log_2 n$ ($n$ элементов вставляется за $\log_2 n$ - по сути все таже сортировка массива)

Стоимость же добавления нового элемента будет уже всего $\log_2 n$ (если дерево сбалансировано), что весьма быстрее, чем в предыдущем способе.

Стоимость же взятия наиболее приоритетного элемента будет равна $\log_2 n$, что уже хуже, чем в предыдущем варианте

In [5]:
std::set<int, std::greater<int> > s1;
for(int i : {5, 2, 7, 9, 3, 1, 6, 8, 11}) s1.insert(i); // построение
/**
* Для примера можно использовать контейнер множества из стандартной библиотеки С++
* Оно базируется на самобалансирующемся красно-черном дереве
**/

In [6]:
std::cout << "Самый приоритетный: " << *s1.begin();

Самый приоритетный: 11

(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fa6da4bd6e0


# Кучи

Куча - это структура данных, которая представляет собой полное бинарное дерево со следующим свойством:

* Значение в родителе больше чем значение в потомках

Таким образом, в корне кучи всегда будет находится самый большой элемент.

<img src='https://upload.wikimedia.org/wikipedia/commons/3/38/Max-Heap.svg'>

Важной особенностью кучи является тот факт, что для ее хранения можно использовать обычный массив следующий образом:

* Если узел имеет индекс i в массиве, то его правый потомок 2i, а левый - 2i+1.
* Родителя также можно определить по индексу - для i-го родитель (i-1)/2.

In [7]:
typedef std::vector<long>::iterator Iter;

In [8]:
Iter left(Iter i, Iter heap_begin, Iter heap_end) { // левый потомок
    Iter l = heap_begin + (i - heap_begin) * 2 + 1;
    return l >= heap_end ? heap_end : l;
}

In [9]:
Iter rigth(Iter i, Iter heap_begin, Iter heap_end) { // правый потомок
    Iter r = heap_begin + (i - heap_begin) * 2 + 2;
    return r >= heap_end ? heap_end : r;
}

In [10]:
Iter parent(Iter i, Iter heap_begin, Iter heap_end) { // родитель
    return heap_begin + (i - heap_begin - 1) / 2;
}

Алгоритм построения кучи из набора очень прост.

Наблюдение:
Пусть у нас есть бинарное дерево, такое что целиком оно не является кучей, однако каждый из потомков корня - являются бинарной кучей. Другими словами свойство кучи нарушается только в корне.

В таком случае, эту структуру можно очень просто превратить в настоящую кучу:

* Выберем максимальный элемент из потомков корня
* Если он больше, чем сам корень, то поменяем их местами
* Теперь свойство кучи, очевидно выполняется в корне - в корне находится самый большой элемент. Однако свойство кучи могло быть нарушено в потомке, с которым мы поменяли корневой элемент. 
* Перейдем в эту вершину и мы получим аналогичную ситуацию - свойство кучи нарушается только в корне. Запустим этот алгоритм на этой вершине.

Таким образом мы будем спускать вершину, пока она не займет необходимое место (возможно в листе). После этой операции полученная структура по построению будет являться бинарной кучей.

In [11]:
void heapify(Iter e, Iter heap_begin, Iter heap_end) { // основная функция для восстановления кучи
    Iter l = left(e, heap_begin, heap_end), r = rigth(e, heap_begin, heap_end); // берем потомков
    Iter largest; 
    if(l != heap_end and *l > *e) largest = l;
    else largest = e; // находим максимальный среди них

    if(r != heap_end and *r > *largest) largest = r;

    if(largest != e) { // если он больше чем тот, что в изначальной вершине
        std::swap(*e, *largest); // меняем их местами
        heapify(largest, heap_begin, heap_end); // выполняем восстановление для потомка
    }
}

#### Построение

Пусть нам дан массив данных. Вторая половина этого массива уже является бинарной кучей - так как это листовые элементы, а значит представляют собой кучи с одним единственным элементом. 

Посмотрим на первый элемент перед ними - на первого родителя листовых элементов. Очевидно, что для него справедлива ситуация, описанная ранее - все потомки явлюятся бинарными кучами (так как листовые элементы), но свойство кучи может наружаться в самой вершине. 

Тогда произведем восстановление кучи, описанное выше. Теперь посмотрим на следующий элемент. Он или опять родитель листов или родитель той вершины, которую мы только что превратили в кучу. Таким образом и для него верна описанная ситуация => восстановим свойство кучи и в этой вершине.

Будет дальше идти по массиву и восстанавливать свойство кучи для элементов.

В конце мы воссстановим свойство кучи для первого элемента, а значит вся структура будет являться полной бинарной кучей.

In [12]:
void build_heap(Iter heap_begin, Iter heap_end) { // построение кучи
    for(Iter i = heap_begin + (heap_end-heap_begin)/2 + 1; i != heap_begin - 1; i--) {
        heapify(i, heap_begin, heap_end); // идем по массиву и восстанавливаем кучу
    }
}

#### Взятие самого приоритетного

Самый приоритетный элемент находится на первом месте по свойству бинарной кучи. Для того, чтобы его удалить, удалим вершину из массива и на его место поставим последний элемент массива. Так как последний элемент массива является листовым, то свойство кучи могло нарушится только в вершине. А мы уже хорошо умеет восстанавливать кучу из этого состояния. 

In [13]:
long Top(std::vector<long> array, Iter heap_begin, Iter heap_end) { // берем самый приоритетный элемент
    if(array.size() == 0) {
        throw std::length_error("Queue is empty");
    }

    long maximum = *heap_begin;
    heap_end--;
    *heap_begin = *heap_end; // берем элемент с конца
    heapify(heap_begin, heap_begin, heap_end); // восстанавливаем кучу

    array.erase(heap_end);

    return maximum;
}

#### Стоимость

С первого взгляда может показаться, что построение кучи также стоит $n \log_2 n$ операций, так как n элементов нужно спустить по бинарному дереву (то есть за $\log_2 n$). Однако на самом деле, эта операция происходит быстрее, так как мы каждый раз работаем не со всем деревом, а только с ее частью:

Так, первый вызов восстановления кучи потребует всего одного действия, так как оно производится над деревом с высотой 1 (его потомки - листья). Далее с деревом с высотой 2 и так далее. Было доказано, что из-за этого стоимость всей операции будет составлять всего $n$ операций! Это гораздо быстрее, чем любой из рассматриваемых ранее подходов.

Стомость взятия самого приоритетного элемента будет стоить $\log_2 n$ - необходимо будет спустить вершину по всему дереву для восстановления кучи.

Добавление также будет стоить $\log_2 n$ - нужно опять спустить вершину по всему дереву.

Такми образом, данная структура является наиболее оптимальный решением для построения очередей с приоритетом.

In [14]:
#include <iostream>
#include <vector>
#include <iterator>
#include <tuple>
#include <stdexcept>
#include <climits>

In [15]:
class PriorityQueue {
public:
    PriorityQueue(){
        retrieve_iters();
    }

    long Max() {
        return *heap_begin;
    }

    long Top() { // берем самый приоритетный элемент
        if(array.size() == 0) {
            throw std::length_error("Queue is empty");
        }

        long maximum = *heap_begin;
        heap_end--;
        *heap_begin = *heap_end; // берем элемент с конца
        heapify(heap_begin); // восстанавливаем кучу

        array.erase(heap_end);
        retrieve_iters();

        return maximum;
    }

    void Insert(long k) { // вставляем ключ
        array.push_back(LONG_MIN);
        retrieve_iters();

        increase_key(heap_end-1, k);
    }

    bool empty() {
        return array.size() == 0;
    }
    
    void build_heap() { // построение кучи
        heap_end = array.end(); 
        for(Iter i = heap_begin + (heap_end-heap_begin)/2 + 1; i != heap_begin - 1; i--) {
            heapify(i); // идем по массиву и восстанавливаем кучу
        }
    }
    
    void setData(std::vector<long> v) {
        array = v;
        retrieve_iters();
    }
    
private:
    void retrieve_iters() { // функция для удобства. Восстанавливает значение итераторов
        heap_begin = array.begin();
        heap_end = array.end();
    }

    Iter left(Iter i) { // левый потомок
        Iter l = heap_begin + (i - heap_begin) * 2 + 1;
        return l >= heap_end ? heap_end : l;
    }

    Iter rigth(Iter i) { // правый потомок
        Iter r = heap_begin + (i - heap_begin) * 2 + 2;
        return r >= heap_end ? heap_end : r;
    }

    Iter parent(Iter i) { // родитель
        return heap_begin + (i - heap_begin - 1) / 2;
    }

    void increase_key(Iter e, long k) { // вспомагательная функция дла вставки
        if(k < *e) {
            throw std::invalid_argument("New key is lesser then current");
        }

        *e = k;
        while(e != heap_begin and *parent(e) < *e) {
            std::swap(*e, *parent(e));
            e = parent(e);
        }
    }

    void heapify(Iter e) { // основная функция для восстановления кучи
        Iter l = left(e), r = rigth(e); // берем потомков
        Iter largest; 
        if(l != heap_end and *l > *e) largest = l;
        else largest = e; // находим максимальный среди них

        if(r != heap_end and *r > *largest) largest = r;

        if(largest != e) { // если он больше чем тот, что в изначальной вершине
            std::swap(*e, *largest); // меняем их местами
            heapify(largest); // выполняем восстановление для потомка
        }
    }

    std::vector<long> array;
    Iter heap_begin;
    Iter heap_end;
};

In [16]:
PriorityQueue q;
q.Insert(5);
q.Insert(7);
q.Insert(3);
q.Insert(10);

std::cout << q.Top() << std::endl;
std::cout << q.Max() << std::endl;
std::cout << q.Top() << std::endl;

std::cout << "----" << std::endl;

q.Insert(125);
q.Insert(1);

std::cout << "----" << std::endl;

while(!q.empty()) {
    std::cout << q.Top() << std::endl;
}

10
7
7
----
----
125
5
3
1


In [17]:
PriorityQueue p;
p.setData(std::vector<long>({5, 2, 7, 9, 3, 1, 6, 8, 11}));
p.build_heap();

while(!p.empty()) {
    std::cout << p.Top() << std::endl;
}

11
9
8
7
6
5
3
2
1
